In [18]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [19]:
import re
import string

# url = 'https://news.v.daum.net/v/20200505000102404'
def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) #변환 후 저장
    full_body = ' '.join(body)
    
    return full_body

# get_detail('https://news.v.daum.net/v/20200505000102404')

In [20]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            NEWS_LIST.to_sql(name = 'NEWS_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료..')

In [21]:
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e)) 

In [22]:
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return NEWS_LIST   


In [23]:
db_delete()

In [24]:
response = requests.get('http://news.daum.net/breakingnews/digital?page=3&regDate=20200505')
root = lxml.html.fromstring(response.content)
for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
    a = li.xpath('div/strong/a')[0]
    url = a.get('href')
    print(url, a.text)

https://v.daum.net/v/20200505193817247 [포스트 코로나] 어제와 달라진 韓, 국제사회 '뉴노멀'된다
https://v.daum.net/v/20200505190832982 LGU+, 상용망서 '5G 단독모드' 테스트 마쳐
https://v.daum.net/v/20200505190517969 타다금지법 헌재로 간다 "행복추구권·평등권 침해"
https://v.daum.net/v/20200505184607746 엘지 벨벳폰, "2년뒤 반납조건 반값" 소비자 묶을까
https://v.daum.net/v/20200505184606744 LG유플러스, 서울서 '5G 단독모드' 테스트 완료
https://v.daum.net/v/20200505184603743 "초유의 온라인개학 가능했던 건 클라우드 덕분이죠"
https://v.daum.net/v/20200505183602568 코로나 사태로 확 뜬 K바이오..해외진출지원사업 대거 선정
https://v.daum.net/v/20200505183601566 [IR52 장영실상] 현대차 / 카파 1.0 T-GDI FFV 엔진
https://v.daum.net/v/20200505183306530 "이 노래 추천해요"..음악플랫폼, AI 기술전쟁
https://v.daum.net/v/20200505183305528 LG유플, 20배 빠른 '5G 단독모드' 서울에서 서비스 테스트 마쳤다
https://v.daum.net/v/20200505183301524 게임 최강국 대한민국, 낯뜨거운 '유저'의식
https://v.daum.net/v/20200505183300521 "2년 뒤 반납하세요" LG 벨벳 40만원대
https://v.daum.net/v/20200505181416222 논란에도 국회 넘은 타다금지법, 헌법재판소로 간다
https://v.daum.net/v/20200505181218189 바닷물 담아두면 식수 되는 물병
https://v.daum.net/v/20200505181216188 플라

In [25]:
page = 1
max_page = 0
dt = '20200721'
response = requests.get(url)
while(True):
    df_list = []
    response = requests.get('http://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                            .format(page, dt))
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        df = pd.DataFrame({'URL' : [url],'TITLE':[a.text],'ARTICLE' : [get_detail(url)]})
        df_list.append(df)   
        
    if df_list:   
        df_10 = pd.concat(df_list)
        db_save(df_10)

    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num = int(a.text)
            if max_page < num:
                max_page = num       
        except:
            pass

    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')

    if (len(span) <= 0) & (page > max_page):
        break
    else:
        page = page + 1


15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..


KeyboardInterrupt: 

In [26]:
NEWS_LIST = db_select()

In [27]:
def save_excel(NEWS_LIST):
    excel = pd.ExcelWriter('뉴스_IT_20200721.xlsx')
    NEWS_LIST.to_excel(excel, '.', index=False )
    excel.save()

In [28]:
save_excel(NEWS_LIST)